<a href="https://colab.research.google.com/github/BlackeWhite/SSD_Modanet/blob/master/Generate_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import File

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#dataset Modanet originale 
fileId = drive.CreateFile({'id': '1FclI6xXIUQfg54Se2w22aCnXOfREcA62'}) 
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(fileId['title'], fileId['id']))
!unzip Moda.zip



# Estrazione delle imagini con annotazioni meno presenti nel dataset 

In [ ]:
import json
import glob
import os

base_path = "./"
path_train = base_path + "modanet2018_instances_train.json"
path_val = base_path + "modanet2018_instances_val.json"

def lista_file_directory(directory,Lista_vuota):
	Lista_vuota=os.listdir(directory) 
	i=0
	for item in Lista_vuota:
		temp=item.split(".")
		Lista_vuota[i]=temp[0]
		i=i+1 
	return Lista_vuota


l_dataset=[]   
l_dataset=lista_file_directory("./train",l_dataset)

data = {}
data["annotations"] = []

lista_occhiali=[]
lista_stivali=[]
lista_cappello=[]
lista_pantaloncini=[]
lista_sciarpa=[]
i=0
with open(path_train, "r") as f:
    lista=json.load(f)

ann = lista["annotations"]
for image in lista["images"]:
    id0=image["id"]
    
    for a in ann:
        if a["image_id"]==id0:
            if a["category_id"]==7:
                lista_occhiali.append(id0)
            elif a["category_id"]==10:
                lista_pantaloncini.append(id0)
            elif a["category_id"]==3:
                lista_stivali.append(id0)
            elif a["category_id"]==13:
                lista_sciarpa.append(id0)   
            elif a["category_id"]==12:
                lista_cappello.append(id0)      
    i+=1
    if i%1000==0:
        print("ne ho fatti 1000 e sono a " + str(i))

d = open("lista_stivali.txt", "w")
for temp in lista_stivali:    
    item=str(temp) + ".jpg\n"
    d.write(item)
d.close()
d = open("lista_cappello.txt", "w")
for temp in lista_cappello:   
    item=str(temp) + ".jpg\n"
    d.write(item)
d.close()   
d = open("lista_sciarpa.txt", "w")
for temp in lista_sciarpa:   
    item=str(temp) + ".jpg\n"
    d.write(item)
d.close()   
d = open("lista_pantaloncini.txt", "w")
for temp in lista_pantaloncini: 
    item=str(temp) + ".jpg\n"
    d.write(item)
d.close()
d = open("lista_occhiali.txt", "w")
for temp in lista_occhiali: 
    item=str(temp) + ".jpg\n"
    d.write(item)
d.close()

In [ ]:
!mkdir sciarpa
!mkdir cappello 
!mkdir pantaloncini 
!mkdir stivali 
!mkdir occhiali

#creazione delle cartelle dove isnerire le immagini con meno annotazioni
import shutil
import os
tld_list=[]
c = open ("lista_sciarpa.txt", "r")
tld_list = [line for line in c]
for item in tld_list:
	item=item.strip('\n')
	if os.path.exists("./train/" + item):
		shutil.move("./train/" + item , "./sciarpa/" + item) 	
tld_list=[]
c.close()
c = open ("lista_cappello.txt", "r")
tld_list = [line for line in c]
for item in tld_list:
	item=item.strip('\n')
	if os.path.exists("./train/" + item):
	 shutil.move("./train/" + item , "./cappello/" + item)
tld_list=[]
c.close()
c = open ("lista_pantaloncini.txt", "r")
tld_list = [line for line in c]
for item in tld_list:
	item=item.strip('\n')
	if os.path.exists("./train/" + item):
		shutil.move("./train/" + item , "./pantaloncini/" + item)
tld_list=[]
c.close()
c = open ("lista_stivali.txt", "r")
tld_list = [line for line in c]
for item in tld_list:
	item=item.strip('\n')
	if os.path.exists("./train/" + item):
		shutil.move("./train/" + item , "./stivali/" + item)    
tld_list=[]
c.close()
c = open ("lista_occhiali.txt", "r")
tld_list = [line for line in c]
for item in tld_list:
	item=item.strip('\n')
	if os.path.exists("./train/" + item):
		shutil.move("./train/" + item , "./occhiali/" + item)      
c.close() 

# Creazione del Dataset 
Train 80% - Validation 10% - Test 10%


In [ ]:
!mkdir new_train
!mkdir new_val 
!mkdir new_test


In [ ]:
import os
import shutil


l_moda_train = []
path="./stivali/" #va fatto anche per le altre 5 cartelle create sopra e per i restanti file 
l_moda=os.listdir(path) 
x=len(l_moda)
train=0.8
val=0.9
Inizio=0
Fine_train=round(x*train)
Fine_val=round(x*val)
print(x)
print(Fine_train)
print(Fine_val)

for y in range(Inizio,Fine_train):
	shutil.move(path + l_moda[y], "./new_train/" + l_moda[y])
for y in range(Fine_train+1,Fine_val):
	shutil.move(path + l_moda[y], "./new_val/" + l_moda[y])
for y in range(Fine_val+1,x):
	shutil.move(path + l_moda[y], "./new_test/" + l_moda[y]) 


# Creazione dei Json contenenti le annotazione per ogni set del Dataset (Train/val/test)

In [ ]:
import json
import glob
import os
from random import randint 
from PIL import Image, ImageDraw

path_train = "./modanet2018_instances_train.json"



def lista_file_directory(directory,Lista_vuota):
    Lista_vuota=os.listdir(directory) 
    i=0
    for item in Lista_vuota:
        temp=item.split(".")
        Lista_vuota[i]=temp[0]
        i=i+1 
    return Lista_vuota

#va fatto per ogni set del dataset (train/val/test)

l_dataset=[]   
l_dataset=lista_file_directory("./new_test",l_dataset) #va fatto per ogni set del dataset (train/val/test)

i=0

data = {}
data["images"] = []
data["type"] = []
data["annotations"] = []
data["categories"] = []


with open(path_train, "r") as f:
    lista=json.load(f)

temp_lista=[]
for x in range(0,len(lista["images"])):
    temp_lista.append(x) 
i=0
prova=True
ann = lista["annotations"]
ann_id0 = []
for image_name in l_dataset:
    id0=0
    image_id_store=0
    for image_id in temp_lista:
        if lista["images"][image_id]["id"]==int(image_name):
            image_id_store=image_id
            temp_lista.remove(image_id)
            id0 = lista["images"][image_id]["id"]
            break    
    data["images"].append(lista["images"][image_id_store])

    if prova==True:
        data["type"].append(lista["type"])
        prova=False


    for a in ann:
        if a["image_id"]==id0:
            ann_id0.append(a)

    i+=1
    if i%1000==0:
        print("ne ho fatti 1000 e sono a " + str(i))

data["annotations"]=ann_id0
data["categories"]=lista["categories"]

with open('annotazioni_test.json', 'a') as outfile:
    json.dump(data, outfile)



ne ho fatti 1000 e sono a 1000
ne ho fatti 1000 e sono a 2000
ne ho fatti 1000 e sono a 3000
ne ho fatti 1000 e sono a 4000
ne ho fatti 1000 e sono a 5000


In [ ]:
!zip annotazioni.zip annotazioni_train.json annotazioni_val.json annotazioni_test.json

  adding: annotazioni_train.json (deflated 74%)
  adding: annotazioni_val.json (deflated 74%)
  adding: annotazioni_test.json (deflated 74%)


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#dataset
Modadat = drive.CreateFile({'id': '1il_eW1COHs1_QijMh3zklvhmE_Dfk0qh'}) 
Modadat.GetContentFile(Modadat['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Modadat['title'], Modadat['id']))
!unzip dataset.zip 
#annotazioni 
annotaz = drive.CreateFile({'id': '1aD4BgDD7grEFYlDKr0k_SHPw1qbt9N1q'}) 
annotaz.GetContentFile(annotaz['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(annotaz['title'], annotaz['id']))
!unzip annotazioni.zip 




# verifiica di tutte le annotazioni per ogni file json del set (in modo da elimianre i file senza annotazioni)

In [ ]:
import os 
import json 

path = "./annotazioni_train.json"

i=0
data = {}
data["images"] = []
data["annotations"] = []


w = open("./train.txt","w")

with open(path, "r") as f:
    lista=json.load(f)
    for image_id in range(0,41908):
        id0 = lista["images"][image_id]["id"]
        ann = lista["annotations"]
        ann_id0 = []
        for a in ann:
            if a["image_id"]==id0:
                ann_id0.append(a)
        if len(ann_id0)==0:      
            w.write("ID=" + str(id0) + " n annotazioni = " + str(len(ann_id0)) +"\n")
        i+=1
        if i%1000==0:
            print("ne ho fatti 1000 e sono a " + str(i))    
w.close()






# conteggio elementi delle varie categorie del dataset

In [ ]:
import json
import glob
import os


#path_train = "/content/annotazioni_test.json"

base_path = "./"
path_train = base_path + "modanet2018_instances_train.json"

data = {}
data["annotations"] = []

lista_bag=0
lista_belt=0
lista_boots=0
lista_footwear=0
lista_outer=0
lista_dress=0
lista_sunglasses=0
lista_pants=0
lista_top=0
lista_shorts=0
lista_skirt=0
lista_headwear=0
lista_scarfetie=0
i=0
with open(path_train, "r") as f:
    lista=json.load(f)

ann = lista["annotations"]
for image in lista["images"]:
    id0=image["id"]
    
    for a in ann:
        if a["image_id"]==id0:
            if a["category_id"]==1:
                lista_bag+=1
            if a["category_id"]==2:
                lista_belt+=1
            elif a["category_id"]==3:
                lista_boots+=1
            elif a["category_id"]==4:
                lista_footwear+=1 
            elif a["category_id"]==5:
                lista_outer+=1                                                                            
            elif a["category_id"]==6:
                lista_dress+=1
            elif a["category_id"]==7:
                lista_sunglasses+=1   
            elif a["category_id"]==8:
                lista_pants+=1  
            elif a["category_id"]==9:
                lista_top+=1                                                          
            elif a["category_id"]==10:
                lista_shorts+=1
            elif a["category_id"]==11:
                lista_skirt+=1        
            elif a["category_id"]==12:
                lista_headwear+=1   
            elif a["category_id"]==13:
                lista_scarfetie+=1     
    i+=1
    if i%1000==0:
        print("ne ho fatti 1000 e sono a " + str(i))

print("Bag: " + str(lista_bag))
print("Belt:" + str(lista_belt))
print("Boots: " + str(lista_boots))
print("Footwear:" + str(lista_footwear))
print("Outer: "+str(lista_outer))
print("Dress: "+str(lista_dress))
print("Sunglasses: "+str(lista_sunglasses))
print("Pants: "+str(lista_pants))
print("Top: "+str(lista_top))
print("Shorts: "+str(lista_shorts))
print("Skirt: "+str(lista_skirt))
print("Headwear: "+str(lista_headwear))
print("Scarf&tie: "+ str(lista_scarfetie))

# Test Tenosrflow 2.0

# Elenco di tutti i file presnti nel set di train e nel set di val (usati per i test delle tensorflow)

In [ ]:
import os

path1="./new_train/"
path2="./new_val/"


lista_train=os.listdir(path1)
lista_val=os.listdir(path2)


t=open("train.txt", 'a')
v=open("val.txt",'a')


for item in lista_train:
  x=item.split(".")
  t.write(x[0] + "\n")
for item in lista_val:
  x=item.split(".")
  v.write(x[0] + "\n")  




# Creazione annotazioni in formato xml per ogni file del dataset estrandole dai file json fatti in precedenza

In [ ]:
import json
import glob
import os
import xml.etree.ElementTree as ET

#va fatto sia per il set di train che per il val 


path_train = "./annotazioni_val.json"
classes = ['background',
           'bag', 'belt', 'boots', 'footwear',
           'outer', 'dress', 'sunglasses', 'pants',
           'top', 'shorts', 'skirt', 'headwear',
           'scarf/tie']

def lista_file_directory(directory,Lista_vuota):
    Lista_vuota=os.listdir(directory) 
    i=0
    for item in Lista_vuota:
        temp=item.split(".")
        Lista_vuota[i]=temp[0]
        i=i+1 
    return Lista_vuota

with open(path_train, "r") as f:
    lista=json.load(f)
   

l_dataset=[]   
l_dataset=lista_file_directory("./new_val",l_dataset) #va fatto per ogni set del dataset 
#l_dataset=lista_file_directory("./new_train",l_dataset)
i=0


temp_lista=[]
for x in range(0,len(lista["images"])):
    temp_lista.append(x) 
i=0



ann = lista["annotations"]
ann_id0 = []
for image_name in l_dataset:
    root = ET.Element('annotation')
    folder = ET.SubElement(root, 'folder')
    folder.text = 'new_train'
    filename = ET.SubElement(root, 'filename')
    filename.text = image_name + ".jpg"
    size = ET.SubElement(root, 'size')
    width = ET.SubElement(size, 'width')
    width.text = '400'
    height = ET.SubElement(size, 'height')
    height.text = '600'
    segmented = ET.SubElement(root, 'segmented')
    segmented.text = '0'


    id0=0
    image_id_store=0
    for image_id in temp_lista:
        if lista["images"][image_id]["id"]==int(image_name):
            image_id_store=image_id
            temp_lista.remove(image_id)
            id0 = lista["images"][image_id]["id"]
            break    


    for a in ann:
        if a["image_id"]==id0:
            object_1 = ET.SubElement(root, 'object')
            category_name = ET.SubElement(object_1, 'name')
            category_name.text = classes[a['category_id']]
            bndbox = ET.SubElement(object_1, 'bndbox')
            xmin = ET.SubElement(bndbox, 'xmin')
            xmin.text = str(a['bbox'][0])
            ymin = ET.SubElement(bndbox, 'ymin')
            ymin.text = str(a['bbox'][1])
            xmax = ET.SubElement(bndbox, 'xmax')
            xmax.text = str(a['bbox'][2])
            ymax = ET.SubElement(bndbox, 'ymax')
            ymax.text = str(a['bbox'][3])
                    
   
 
    tree = ET.ElementTree(root)
    tree.write("./annotazioni_val/"+str(image_name)+".xml", pretty_print=True)
    i+=1
    if i%1000==0:
        print("ne ho fatti 1000 e sono a " + str(i))

 

ne ho fatti 1000 e sono a 1000
ne ho fatti 1000 e sono a 2000
ne ho fatti 1000 e sono a 3000
ne ho fatti 1000 e sono a 4000
ne ho fatti 1000 e sono a 5000


In [ ]:
!zip annotazioni_xml.zip annotazioni_train/* annotazioni_val/*

In [ ]:
# https://gist.github.com/iKhushPatel/ed1f837656b155d9b94d45b42e00f5e4

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[1][0].text),
                     int(member[1][1].text),
                     int(member[1][2].text),
                     int(member[1][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

#inserisce in un unico csv tutti i file xml del train (analogo per il validation)
image_path = '/content/annotazioni_train'
xml_df = xml_to_csv(image_path)
xml_df.to_csv('./train_labels.csv', index=None)

image_path = '/content/annotazioni_val'
xml_df = xml_to_csv(image_path)
xml_df.to_csv('./val_labels.csv',index=None)



In [ ]:
#https://gist.github.com/iKhushPatel/5614a36f26cf6459cc49c8248e8b5b48
#creazione dei file tfrecords per train e val 

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
import sys

from models.research.object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'bag':
        return 1
    elif row_label == 'belt':
        return 2
    elif row_label == 'boots':
        return 3
    elif row_label == 'footwear':
        return 4
    elif row_label == 'outer':
        return 5
    elif row_label == 'dress':
        return 6
    elif row_label == 'sunglasses':
        return 7
    elif row_label == 'pants':
        return 8
    elif row_label == 'top':
        return 9
    elif row_label == 'shorts':
        return 10    
    elif row_label == 'skirt':
        return 11
    elif row_label == 'headwear':
        return 12
    elif row_label == 'scarf/tie':
        return 13                                              
    else:
        return 0


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example



writer = tf.io.TFRecordWriter('./val.record')
path = os.path.join('/content/new_val')
examples = pd.read_csv('/content/val_labels.csv')
grouped = split(examples, 'filename')
for group in grouped:
  tf_example = create_tf_example(group, path)
  writer.write(tf_example.SerializeToString())

        

writer.close()
output_path = './val.record'
print('Successfully created the TFRecords: {}'.format(output_path))


Successfully created the TFRecords: ./val.record
